# Part 1

#### Import pandas and read in table.

In [1]:
import pandas as pd

In [2]:
lst = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
postal = lst[0]

In [3]:
postal.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Cleaning the postal code dataframe.

In [4]:
postal.drop(postal[postal['Borough'] == 'Not assigned'].index, inplace=True)
postal.replace(postal[postal['Neighborhood']=='Not assigned'], postal['Borough'], inplace=True)
postal.replace(regex=[r' /'], value=',', inplace=True)
postal.reset_index(drop = True, inplace=True)
postal.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
postal.shape

(103, 3)

# Part 2

#### Reading geospatial data.

In [6]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Cleaning up the data and combining the geospatial dataframe with the postal code dataframe.

In [7]:
geo.rename(columns={"Postal Code": "Postal code"}, inplace=True)
geo_postal = pd.concat([postal, geo], axis=1, join='inner')
geo_postal.reset_index(drop = True, inplace = True)

In [8]:
geo_postal.head()

,Postal code,Borough,Neighborhood,Postal code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
